### Import all relevant dependencies

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from pathlib import Path
import yaml
from oed.experiments.experiment_library.latin_hypercube import LatinHypercube
from src.experiments.experiment_library.fcs_a_design import FCSADesign
from src.experiments.experiment_library.fcs_d_design import FCSDDesign
from src.experiments.experiment_library.fcs_pi_design import FCSPiDesign
from src.math_utils.blackbox_evaluation import evaluate_blackbox_region
from src.minimizer.minimizer_library.differential_evolution_parallel import DifferentialEvolutionParallel
from src.statistical_models.statistical_model_library.fcs_gaussian_noise_model import FCSGaussianNoiseModel
from src.utils.experiment_serialization import save_experiment_results
from src.visualization.plotting_functions import *
from src.math_utils.experiment_metrics import calculate_estimator_metrics
from src.math_utils.derivatives.numeric_derivative_calculator import NumericDerivativeCalculator
from src.model.hahn_stack_model import HahnStackModel
from src.math_utils.scaler.hahn_parameter_scaler import HahnParameterScaler
from src.model.parameter_set.hahn_parameter_set import HahnParameterSet
from src.utils.experiment_serialization import import_experiment_results

### Define the Experiment Metadata
- bounds for the free parameters
- operating conditions
- experiment repetitions
- true parameter set
- Experiment variance
- current values

Naming:
unscaled/ rescaled: actual physical values
scaled: scaled value between 0 & 1

In [2]:
path = "../data/config/reduced_example_config_workflow.yaml"

with open(path, "r") as f:
    config = yaml.safe_load(f)

number_designs = config["number_designs"] #Amount of LH Designs
n_rep = config["n_rep"] #Amount of experiment repetitions
n_current_values = config["n_current_values"] #Amount of individual current values
sigma = config["sigma"] #experiment variance (10mV variance in repeated experiments)

# lower and upper bounds for operating conditions
upper_bounds_operating_conditions = np.array(config["upper_bounds_operating_conditions"])
lower_bounds_operating_conditions = np.array(config["lower_bounds_operating_conditions"])

I_S_array = np.linspace(1, 600, n_current_values) # initialize applicable current array request

# select parameter values and names to be analyzed
unscaled_theta_true = np.array([list(HahnParameterSet().free_parameters.values())[i] for i in [1, 2, 4]])
names_theta = [list(HahnParameterSet().free_parameters.keys())[i] for i in [1, 2, 4]]

# initialize lower and upper bounds for free parameter values
unscaled_upper_bounds_free_params = np.array(config["unscaled_upper_bounds_free_params"], dtype=float)
unscaled_lower_bounds_free_params = np.array(config["unscaled_lower_bounds_free_params"], dtype=float)

print(names_theta, unscaled_lower_bounds_free_params, unscaled_theta_true, unscaled_upper_bounds_free_params)


['j_0_ref', 'r_el', 'D_GDL_ref'] [1.e+01 1.e-06 1.e-06] [2.1308e+03 4.2738e-06 8.6266e-06] [5.e+03 1.e-03 1.e-05]


### Define Scaler and scale parameter values as well as  bounds of operating conditions and parameters
Scalers are saved in variable "scaler" and handed over to stack model.

In [3]:
scaler = HahnParameterScaler() # define scaler

# Stack bounds of free parameters
free_parameter_bounds = np.vstack([
    unscaled_lower_bounds_free_params,
    unscaled_upper_bounds_free_params
]).T

# Stack operating condition bounds (rows = condition, columns = [min, max])
operating_condition_bounds = np.vstack([
    lower_bounds_operating_conditions,
    upper_bounds_operating_conditions
]).T

# Determine current range for scaling
current_bounds = np.array([[I_S_array.min(), I_S_array.max()]])

scaled_theta_true = scaler.scale_theta(unscaled_theta_true, free_parameter_bounds)

# scale bounds of operating conditions to hand over for Experimental designs incl. LHC
scaled_upper_bounds = scaler.scale_params(upper_bounds_operating_conditions, operating_condition_bounds)
scaled_lower_bounds = scaler.scale_params(lower_bounds_operating_conditions, operating_condition_bounds)

# scale bounds of free parameters for initializing Model
scaled_lower_bounds_theta, _ = scaler.scale(unscaled_lower_bounds_free_params, free_parameter_bounds)
scaled_upper_bounds_theta, _ = scaler.scale(unscaled_upper_bounds_free_params, free_parameter_bounds)

# Print scaling results of scaled true parameters
print("Scaled theta:", scaled_theta_true)
print("Rescaled theta:", unscaled_theta_true)

Scaled theta: [0.42501002 0.00327708 0.8474    ]
Rescaled theta: [2.1308e+03 4.2738e-06 8.6266e-06]


### Define the parametric function including handover of bounds

In [4]:
hahn_fc_model = HahnStackModel(parameter_set=HahnParameterSet(free_parameters=names_theta)) #Instantiate the generic Hahn Model
calculator = NumericDerivativeCalculator(hahn_fc_model, scaler) #Instanciate the derivation calculator function

# Initialize statistical model function
statistical_model = FCSGaussianNoiseModel(model_function=hahn_fc_model,
                                          der_function=calculator,
                                          lower_bounds_x=scaled_lower_bounds,
                                          upper_bounds_x=scaled_upper_bounds,
                                          lower_bounds_theta=scaled_lower_bounds_theta,
                                          upper_bounds_theta=scaled_upper_bounds_theta,
                                          sigma=sigma,
                                          scaler = scaler,)

# initialize blackbox function returning noised experiment results
def blackbox_model(x):
    return statistical_model.random(theta=scaled_theta_true, x=x)

### Import LHC Designs

In [5]:
import_path = ".." / Path("data") / "experimental_designs" / "lhc"
df_exp_input = import_experiment_results(import_path/"lhc.csv")

# prepare LHC data for further analysis
x_designs = df_exp_input[['Pressure', 'Temperature', 'Stoichiometry', 'Current']].to_numpy()
y_designs = df_exp_input[['Voltage']].to_numpy()

# Slice Designs (n=25000) of the repetitions (n=250) for optimal experiment calculation
x0_LH_design = x_designs[:number_designs * n_current_values:]
print(len(x0_LH_design))

50


### Import estimated Thetas

In [6]:
scaled_estimated_thetas = np.loadtxt(import_path/"estimated_thetas.csv", delimiter=",")

# calculate metrics and mean theta value
estimated_theta, _, _, _ = calculate_estimator_metrics(scaled_estimated_thetas, scaled_theta_true)
unscaled_estimated_theta = scaler.rescale_theta(estimated_theta)
print('The estimated Theta mean for all estimations is \n', unscaled_estimated_theta)

The estimated Theta mean for all estimations is 
 [2.12822360e+03 4.28510359e-06 8.75152321e-06]


### Initialize Minimizer function

In [7]:
parameter_index = config["parameter_index"] # individual parameter from parameter set that shall be evaluated in the Pi-Design
number_new_designs = 5 #["number_new_designs"] # number of new designs
iterations = 1000 # maximum iterations of minimizer function

minimizer = DifferentialEvolutionParallel(maxiter=iterations, n_workers=-1, display=True)

### Initialize and execute calculation of different experimental designs

In [8]:
# calculate additional LHC Experiments
LH_new = LatinHypercube(lower_bounds_design=lower_bounds_operating_conditions,
                        upper_bounds_design=upper_bounds_operating_conditions,
                        number_designs=number_new_designs)

print(LH_new.experiment)

[[2.20192314e+05 3.53636223e+02 1.65682874e+00]
 [2.99474932e+05 3.60000576e+02 2.43260634e+00]
 [1.72588507e+05 3.31661428e+02 2.07640243e+00]
 [2.05391498e+05 3.25140274e+02 3.33497991e+00]
 [3.21532514e+05 3.40764616e+02 2.86787819e+00]]


In [ ]:
# calculate additional Pi-Designs underestimated theta and the chosen parameter index
pi_design = FCSPiDesign(number_designs=number_new_designs,
                        lower_bounds_design=lower_bounds_operating_conditions,
                        upper_bounds_design=upper_bounds_operating_conditions,
                        initial_theta=estimated_theta,
                        statistical_model=statistical_model,
                        previous_experiment=x0_LH_design,
                        minimizer=minimizer,
                        index=parameter_index)

print(pi_design.experiment)

Calculating the pi...


In [ ]:
# calculate additional D-Designs under estimated theta
d_design = FCSDDesign(number_designs=number_new_designs,
                      lower_bounds_design=lower_bounds_operating_conditions,
                      upper_bounds_design=upper_bounds_operating_conditions,
                      initial_theta=estimated_theta,
                      statistical_model=statistical_model,
                      previous_experiment=x0_LH_design,
                      minimizer=minimizer, )

print(d_design.experiment)

In [ ]:
# calculate additional D-Designs under estimated theta
a_design = FCSADesign(number_designs=number_new_designs,
                      lower_bounds_design=lower_bounds_operating_conditions,
                      upper_bounds_design=upper_bounds_operating_conditions,
                      initial_theta=estimated_theta,
                      statistical_model=statistical_model,
                      previous_experiment=x0_LH_design,
                      minimizer=minimizer, )

print(a_design.experiment)

### Save all experimental designs as individual CSV files

In [ ]:
# the experiments per se have 5 entries, exported should be all experiments with evaluation, so: new_designs * n_rep * n_current_values = number of lines
experiments = [LH_new, a_design, d_design, pi_design]
experiment_names = ["LH_new", "a_design", "d_design", "pi_design"]

root = Path.cwd().parent   # this will be oed_fuel_cell_model/
data_path = root / "data" / "experimental_designs" / "other"
data_path.mkdir(parents=True, exist_ok=True)

for experiment, name in zip(experiments, experiment_names):
    evaluation_experiment, x0_design = evaluate_blackbox_region(blackbox_model, experiment.experiment, I_S_array, repetitions=n_rep)
    x_array = np.array(x0_design)
    eval_array = np.array(evaluation_experiment).reshape(-1, 1)
    combined_array = np.hstack((x_array, eval_array))
    print(f"Experiment: {name} Length (n_exp*n_rep*n_current_values):{len(combined_array)}")
    file_path = data_path / f"{name}.csv"
    save_experiment_results(combined_array, file_path)

### Plot all calculated operating conditions for additionally calculated experiments

In [ ]:
opCons = ["Pressure [kPa]", "Temperature [K]", "Stoichiometry [-]"]

plot_experiment_matrix([LH_new.experiment, a_design.experiment, d_design.experiment, pi_design.experiment], opCons, exp_labels=["LH_new", "a_design", "d_design", "pi_design"])

In [ ]:
LH = x0_LH_design[::10]
LH = [row[:3] for row in LH]

In [ ]:
opCons = ["Pressure [kPa]", "Temperature [K]", "Stoichiometry [-]"]

plot_experiment_matrix([LH, LH_new.experiment], opCons, exp_labels=["LH", "LH_new"])

In [ ]:
opCons = ["Pressure [kPa]", "Temperature [K]", "Stoichiometry [-]"]

plot_experiment_matrix([LH, a_design.experiment], opCons, exp_labels=["LH", "a_design"])

In [ ]:
opCons = ["Pressure [kPa]", "Temperature [K]", "Stoichiometry [-]"]

plot_experiment_matrix([LH, d_design.experiment], opCons, exp_labels=["LH", "d_design"])

In [ ]:
opCons = ["Pressure [kPa]", "Temperature [K]", "Stoichiometry [-]"]

plot_experiment_matrix([LH, pi_design.experiment], opCons, exp_labels=["LH", "pi_design"])